### `VocabParallelCrossEntropy`

In [ ]:
import torch
from torch.autograd import Function

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
rank = 4
world_size = 100

In [ ]:
# def extract_vocab_range(num_embeddings, rank, world_size):
#     # per_patrition_vocab_size = num_embeddings // world_size
#     start_idx = rank * num_embeddings
#     end_idx = start_idx + num_embeddings
#     return start_idx, end_idx

def extract_vocab_range(
    per_partition_vocab_size: int, rank, world_size: int
):
    index_f = rank * per_partition_vocab_size
    index_l = index_f + per_partition_vocab_size
    return index_f, index_l

In [ ]:
# extract_vocab_range(num_embeddings=100, rank=rank, world_size=world_size)

In [ ]:
class VocabParallelCrossEntropy(Function):
    @staticmethod
    def forward(ctx, vocab_parallel_logits, target):
        logits_max = torch.max(vocab_parallel_logits, dim=-1)[0]
        set_trace()
        
        # TODO: why max two times?
        # torch.distributed.all_reduce(
        #     logits_max,
        #     op=torch.distributed.ReduceOp.MAX
        # )
        
        # TODO: ?
        vocab_parallel_logits.sub_(logits_max.unsqueeze(dim=-1))

In [ ]:
# logits = torch.randn(100, 50)
# targets = torch.randint(low=0, high=100, size=(4, 50))
vocab_parallel_logits = torch.range(0,7).repeat(16,4)
target = torch.arange(0,32,2)

/var/folders/pf/g3nr86yn4j71vzzmv6knwdhr0000gp/T/ipykernel_11507/3761836215.py:3: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  vocab_parallel_logits = torch.range(0,7).repeat(16,4)


In [ ]:
target.shape, vocab_parallel_logits.shape

(torch.Size([16]), torch.Size([16, 32]))

In [ ]:
partition_vocab_size = vocab_parallel_logits.size()[-1]

In [ ]:
logits.size()[-1]

50

In [ ]:
logits_max = torch.max(vocab_parallel_logits, dim=-1)[0]

In [ ]:
logits_max.shape

torch.Size([16])

In [ ]:
logits_max.unsqueeze(dim=-1).shape

torch.Size([16, 1])

In [ ]:
logits.shape

torch.Size([4, 50])

In [ ]:
embedding_dim

50

In [ ]:
vocab_start_idx, vocab_end_idx = extract_vocab_range(partition_vocab_size, rank, world_size)

In [ ]:
vocab_start_idx, vocab_end_idx

(128, 160)

In [ ]:
target

tensor([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30])

In [ ]:
target_mask = (target < vocab_start_idx) | (target >= vocab_end_idx)

In [ ]:
target_mask

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [ ]:
masked_target = target.clone() - vocab_start_idx

In [ ]:
masked_target

tensor([-128, -126, -124, -122, -120, -118, -116, -114, -112, -110, -108, -106,
        -104, -102, -100,  -98])